The notebook contains code to extract clonotype frequency statistics from raw sequencing paired-reads [data](https://www.ebi.ac.uk/ena/browser/view/PRJEB38339) 
used in ["Use of machine learning to identify a T cell response to SARS-CoV-2"](https://www.sciencedirect.com/science/article/pii/S2666379121000033) (Shoukat et al.)
with
[MiXCR](https://github.com/milaboratory/mixcr).

Records with aliases containing "TRB" or "TCRb" were used.

This notebook should be moved to the uncompressed folder. The folder also contains subdirectories for each chosed record; each subdirectory contains 2 gzipped `.fastq` files with forward and reversed reads.

MiXCR command to process the reads is as follows:
```bash
mixcr analyze amplicon --species hs \
        --starting-material dna \
        --5-end v-primers \
        --3-end j-primers \
        --adapters adapters-present \
        --receptor-type TRB \
        R1.fastq R2.fastq results
```

In [4]:
import os
import subprocess
import shlex
import re
from copy import copy

# MiXCR processing

In [112]:
# patterns to extract subject number from filename
pattient_id_pattern = re.compile(r'Pt-[0-9]+-[0-9]+')  
healthy_id_pattern = re.compile(r'HD[0-9]+')

s = """mixcr analyze amplicon --species hs \
        --starting-material dna \
        --5-end v-primers \
        --3-end j-primers \
        --adapters adapters-present \
        --receptor-type TRB """

template_command = shlex.split(s)

path = './'
for dirname in tqdm(os.listdir(path)):
    reads = {1: None, 2: None}
    for filename in os.listdir(path+dirname):
        if '_R1_' in filename:
            reads[1] = filename
        else:
            reads[2] = filename
            
    if all(group not in reads[1] for group in ['HD', 'Pt']):
        continue
    
    # don't need B cells
    if 'IGH' in reads[1]:
        continue

    m = re.search(pattient_id_pattern, reads[1])
    if not m:  # didn't match
        m = re.search(healthy_id_pattern, reads[1])  
    
    res_filename = m.group(0)
    
    command = copy(template_command)
    command.extend([reads[1], reads[2], res_filename])
    
    # run bash 
    subprocess.run(command, cwd=path+dirname)

100%|██████████| 240/240 [2:06:19<00:00, 31.58s/it]  


In [134]:
# move files to group-specific folder
healthy_path = '../healthy/'
patient_path = '../patient/'

path = './'
for dirname in os.listdir(path):
    for filename in os.listdir(path + dirname):
        if filename.endswith('.txt'):
            
            command = ['cp', f'{path + dirname}/{filename}']
            
            if 'Pt' in filename:
                command.append(patient_path)
            else:
                command.append(healthy_path)
            
            subprocess.run(command, check=True)        